<a href="https://colab.research.google.com/github/Yuni-J/dataKakao/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01 카카오톡 데이터 전처러


##구글 드라이브와 코랩 연결하기 (구글 드라이브 마운트)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd

## 폴더 경로
f_path = '/content/drive/MyDrive/KakaoTalk_data'

## 파일 경로
files = {
    'katalk_friend.txt'
}

##정규 표현식 사용

\-+ d{4}년 \d{1,2}월 \d{1,2}일 [월화수목금토일]요일 -+

- -+ : “-”가 하나 이상 반복  (앞뒤의 긴 하이픈)
- \d{4}년 : 연도 (4자리 숫자 + “년)
- \d{1,2}월 : 월 (1~2자리 숫자 + “월”)
- \d{1,2}일 : 일 (1~2자리 숫자 + “일”)
- -+ : "-"가 하나 이상 반복 (뒤쪽 하이픈)


\[\S+\] \[오\S+ \d{1,2}:\d{2}\]\s+(.+)

- \[\S+\]  : 대괄호 안에 공백이 없는 문자열
- \[오\S+ \d{1,2}:\d{2}\]\s :
- - 오\S+ : “오”가 들어가는 공백이 없는 문자열
- - 공백
- -  \d{1,2}:\d{2} : 시간 형식
- (.+) : 내용 추출


In [20]:
import re

text = """--------------- 2022년 1월 16일 일요일 ---------------
[민지이잉] [오전 12:02] ㅋㅋㅋㅋㅋㅋㅋㅋㄱㅇㄱㅋㅋㅋㅋㅋ진짴ㅋㅋ
[민지이잉] [오전 12:02] 다들 잘쟈!!!
[윤희] [오전 12:04] 구랭
[윤희] [오전 12:04] 잘쟈"""

# 날짜 패턴
date_pattern = r"-+ (\d{4}년 \d{1,2}월 \d{1,2}일 [월화수목금토일]요일) -+"

# 메시지 패턴
message_pattern = r"\[([^\]]+)\] \[(오전|오후) (\d{1,2}:\d{2})\] (.+)"

# 날짜 찾기
dates = re.findall(date_pattern, text)

# 메시지 찾기
matches = re.finditer(message_pattern, text)

current_date_index = 0  # 처음에는 첫 번째 날짜부터 시작

for match in matches:
    name = match.group(1)
    am_pm = match.group(2)
    time = match.group(3)
    content = match.group(4)

    # 날짜가 바뀌면 current_date_index를 업데이트하여 새로운 날짜를 출력
    while current_date_index < len(dates) - 1 and match.start() > text.find(dates[current_date_index + 1]):
        current_date_index += 1

    current_date = dates[current_date_index] if current_date_index >= 0 else "날짜 없음"

    print(f"날짜: {current_date}")
    print(f"사람: {name}")
    print(f"시간: {am_pm} {time}")
    print(f"내용: {content}")
    print('-' * 40)


날짜: 2022년 1월 16일 일요일
사람: 민지이잉
시간: 오전 12:02
내용: ㅋㅋㅋㅋㅋㅋㅋㅋㄱㅇㄱㅋㅋㅋㅋㅋ진짴ㅋㅋ
----------------------------------------
날짜: 2022년 1월 16일 일요일
사람: 민지이잉
시간: 오전 12:02
내용: 다들 잘쟈!!!
----------------------------------------
날짜: 2022년 1월 16일 일요일
사람: 윤희
시간: 오전 12:04
내용: 구랭
----------------------------------------
날짜: 2022년 1월 16일 일요일
사람: 윤희
시간: 오전 12:04
내용: 잘쟈
----------------------------------------


## 카카오톡 데이터 파싱하기

In [22]:
import re
import pandas as pd

def katalk_msg_parse(file_path):
  my_katalk_data = list()
  katalk_msg_pattern = message_pattern
  date_info = date_pattern
  current_date = None

  for line in open(file_path, encoding='utf-8'):
    date_match = re.search(date_info, line)
    if date_match:
      current_date = date_match.group(1)
      continue

    message_match = re.search(katalk_msg_pattern, line)
    if message_match:
      name = message_match.group(1)
      am_pm = message_match.group(2)
      time = message_match.group(3)
      content = message_match.group(4)

      if current_date:
        my_katalk_data.append({
            'date' : current_date,
            'user_name' : name,
            'time' : f"{am_pm} {time}",
            'text' : content
        })

      elif len(my_katalk_data) > 0:
        my_katalk_data[-1]['text'] += '\n' + line.strip()

  my_katalk_data = pd.DataFrame(my_katalk_data)

  return my_katalk_data


